In [1]:
import pandas as pd
import plotly.express as px
from dash import dcc, html, Dash
from dash.dependencies import Input, Output

# https://www.kaggle.com/datasets/sachinsarkar/climate-change-global-temperature-data?resource=download
df = pd.read_csv('GlobalLandTemperaturesByCountry.csv')

# Convert 'dt' column to datetime and extract year
df['dt'] = pd.to_datetime(df['dt'])
df['Year'] = df['dt'].dt.year

# Drop rows with missing temperature values
df = df.dropna(subset=['AverageTemperature'])

# Calculate average temperature per year per country
avg_temp_per_year = df.groupby(['Country', 'Year'])['AverageTemperature'].mean().reset_index()

# Calculate average yearly temperature for each country
avg_temp_change = avg_temp_per_year.groupby('Country')['AverageTemperature'].mean().reset_index()

# Calculate the change in average temperature from the first to the last year recorded
first_last_temp = avg_temp_per_year.groupby('Country')['AverageTemperature'].agg(['first', 'last']).reset_index()
first_last_temp['TempChange'] = first_last_temp['last'] - first_last_temp['first']

# Get the first and last year for each country
first_last_temp['FirstYear'] = avg_temp_per_year.groupby('Country')['Year'].min().values
first_last_temp['LastYear'] = avg_temp_per_year.groupby('Country')['Year'].max().values

# Add first and last temperatures to the DataFrame
first_last_temp['FirstTemp'] = avg_temp_per_year.groupby('Country')['AverageTemperature'].first().values
first_last_temp['LastTemp'] = avg_temp_per_year.groupby('Country')['AverageTemperature'].last().values

# Initialize the Dash app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1('Global Temperature Data Dashboard'),

    # Dropdown to select country
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in avg_temp_change['Country']],
        value='United States',  # Default country
        multi=False,
        style={'width': '50%'}
    ),

    # Line chart to display temperature trends
    dcc.Graph(id='temperature-line-chart'),

    # World map to display average yearly temperature
    dcc.Graph(
        id='temperature-change-map',
        figure=px.choropleth(avg_temp_change,
                             locations='Country',
                             locationmode='country names',
                             color='AverageTemperature',
                             color_continuous_scale=px.colors.sequential.Turbo,
                             range_color=[-20, 25],
                             title='Average Yearly Temperature by Country')
    ),

    # New map for change in average yearly temperature
    dcc.Graph(
        id='temperature-change-range-map',
        figure=px.choropleth(first_last_temp,
                             locations='Country',
                             locationmode='country names',
                             color='TempChange',
                             color_continuous_scale=px.colors.sequential.Sunset,
                             range_color=[-2, 12],
                             title='Change in Average Yearly Temperature ',
                             hover_name='Country',
                             hover_data=['FirstYear', 'FirstTemp', 'LastYear', 'LastTemp'])
    )
])

# Callback function to update line chart based on selected country
@app.callback(
    Output('temperature-line-chart', 'figure'),
    Input('country-dropdown', 'value')
)
def update_graph(selected_country):
    # Filter data based on selected country
    filtered_df = avg_temp_per_year[avg_temp_per_year['Country'] == selected_country]
    
    # Create the line chart
    fig = px.line(filtered_df, x='Year', y='AverageTemperature',
                  title=f'Average Temperature Trends in {selected_country}',
                  labels={'Year': 'Year', 'AverageTemperature': 'Average Temperature (°C)'})
    
    # Customize chart layout
    fig.update_layout(transition_duration=500)
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
